In [1]:
import pandas as pd 
from adtk.visualization import plot
from adtk.detector import QuantileAD


data_import = pd.read_csv("data-20.csv" , index_col= "periodname", parse_dates= True) 
data_import.head() 

/var/folders/dl/jbncfxws79vf31sx5dgvkh140000gn/T/ipykernel_7340/4291433731.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_import = pd.read_csv("data-20.csv" , index_col= "periodname", parse_dates= True)


,periodid,periodcode,perioddescription,organisationunitid,organisationunitname,organisationunitcode,organisationunitdescription,105-EP01b. Malaria Total,105-EP01c. Malaria Confirmed (B/s and RDT Positive),105-EP01d. Malaria cases treated,105-MC04. Malaria in pregnancy,105-PS01a. Malaria Microscopy - Done,105-PS01b. Malaria Microscopy - Positive,105-PS02a. Malaria RDTs - Done,105-PS02b. Malaria RDTs - Positive,105-SS11a. Malaria Rapid Diagnostic tests - Quantity consumed,105-SS11b. Malaria Rapid Diagnostic tests - Days out of stock,105-SS11c. Malaria Rapid Diagnostic tests - Stock on hand,105-SS11d. Malaria Rapid Diagnostic tests - Quantity Expired
periodname,,,,,,,,,,,,,,,,,,,
2023-03-01,202303,202303,NaN,auzuV39xOTU,5th Military Division Health Centre IV,NaN,NaN,114.0,114.0,114.0,NaN,485.0,69.0,114.0,45.0,300.0,0.0,800.0,0.0
2023-03-01,202303,202303,NaN,b2LlvIFGc5U,AAR Gulu Clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-01,202303,202303,NaN,aGfndLfPcOz,Abilonino Health Centre II,NaN,NaN,173.0,173.0,173.0,0.0,NaN,NaN,285.0,173.0,3000.0,0.0,2575.0,0.0
2023-03-01,202303,202303,NaN,JryqknC2SNp,Abwoch Health Centre III,NaN,NaN,176.0,176.0,176.0,NaN,12.0,7.0,308.0,164.0,700.0,0.0,1925.0,0.0
2023-03-01,202303,202303,NaN,aaEtUWpmosm,Acet Community Oriented Clinic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
import pandas as pd
from adtk.visualization import plot
from adtk.detector import QuantileAD

def detect_outliers(data, column_name):


    data = data[[column_name, 'organisationunitname']].copy()

    # Placeholder for the results
    outlier_results = []

    for unit in data['organisationunitname'].unique():
        print('processing unit:', unit)

        sample_fac_data = data[data['organisationunitname'] == unit]
        period_names = sample_fac_data.index

        try:
            # Data validation and imputation
            data_test = sample_fac_data.drop('organisationunitname', axis=1)
            data_test = validate_series(data_test)


            quantile_ad = QuantileAD(high=0.99, low=0.01)
            #anomalies = quantile_ad.fit_detect(data_test)
            anomaly_scores = quantile_ad.fit_detect(data_test) 
            # Create results DataFrame
            anomalies = pd.DataFrame(index=period_names)
            anomalies['organisationunitname'] = unit
            anomalies[column_name] = data_test[column_name]  
            #anomalies['outlier'] = outlier_flags
            anomalies['outlier'] = anomaly_scores != 0

            outlier_results.append(anomalies)

        except Exception as e:
            print(f"Error during processing for unit {unit}: {e}. Skipping this unit.")
            continue

    # Concatenate results
    all_outliers = pd.concat(outlier_results, ignore_index=False)
    return all_outliers

def validate_series(data_test):
    """Validates the data and handles missing values.
    """
    # Other validation checks (ensure data type, etc.)
    data_test = data_test.fillna(data_test.mean())  # Impute missing values with the mean
    return data_test


#  --- Example usage ---
# Assuming you have loaded your data into a DataFrame named 'df'
outliers_df = detect_outliers(data_import.copy() , '105-EP01c. Malaria Confirmed (B/s and RDT Positive)')  # Make a copy to avoid modifying the original data
print(outliers_df)


processing unit: 5th Military Division Health Centre IV
processing unit: AAR Gulu Clinic
processing unit: Abilonino Health Centre II
processing unit: Abwoch Health Centre III
processing unit: Acet Community Oriented Clinic
Error during processing for unit Acet Community Oriented Clinic: Valid values are not enough for training.. Skipping this unit.
processing unit: Acet Health Centre III
processing unit: Acholi-Bur Health Centre III
processing unit: Acholpii Health Centre III
processing unit: Acuru Health Centre II
processing unit: Adilang Health Centre III
processing unit: Agonga Karine Community Health Centre II
processing unit: Agoro Health Centre III
processing unit: Akilok Health Centre II
processing unit: Akuna Laber Health Centre III
processing unit: Akurumor Health Centre II
processing unit: Akworo Health Centre III
processing unit: Alero Health Centre III
processing unit: Alim Health Centre II
processing unit: Alokolum Health Centre II
processing unit: Alop Health Centre II
pr

processing unit: Lukwir Health Centre II
processing unit: Lukwor Health Centre II
processing unit: Lulyango Health Centre II
processing unit: Lututuru Prisons Health Centre II
processing unit: Madi Kiloch Health Centre II
processing unit: Madi-Opei Health Centre IV
processing unit: Mama Chef Clinic
Error during processing for unit Mama Chef Clinic: Valid values are not enough for training.. Skipping this unit.
processing unit: Marie Stopes Uganda Gulu Health Centre II
processing unit: Mary Immaculate Health Centre II
processing unit: Mary Queen Of Peace Health Centre III
processing unit: Maternity Home Health Centre II
Error during processing for unit Maternity Home Health Centre II: Valid values are not enough for training.. Skipping this unit.
processing unit: Meeting Point Health Centre II
processing unit: Minakulu (Bobi) Health Centre II
processing unit: Mola Medical Centre Clinic
Error during processing for unit Mola Medical Centre Clinic: Valid values are not enough for training.

processing unit: Victory Medical Centre Clinic (Laroo)
processing unit: Vision Medical Centre
Error during processing for unit Vision Medical Centre: Valid values are not enough for training.. Skipping this unit.
processing unit: Wiianaka Health Centre II
processing unit: Wipolo Health Centre II
processing unit: Wol Health Centre III
processing unit: Yot Kom Health Centre III
                              organisationunitname  \
periodname                                           
2023-03-01  5th Military Division Health Centre IV   
2023-04-01  5th Military Division Health Centre IV   
2023-05-01  5th Military Division Health Centre IV   
2023-06-01  5th Military Division Health Centre IV   
2023-07-01  5th Military Division Health Centre IV   
...                                            ...   
2023-10-01               Yot Kom Health Centre III   
2023-11-01               Yot Kom Health Centre III   
2023-12-01               Yot Kom Health Centre III   
2024-01-01               Yo

In [3]:
#outlier_df.head
outliers_df.to_csv("sample_results.csv")

In [4]:
data_import.info() 

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4188 entries, 2023-03-01 to 2024-02-01
Data columns (total 19 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   periodid                                                       4188 non-null   int64  
 1   periodcode                                                     4188 non-null   int64  
 2   perioddescription                                              0 non-null      float64
 3   organisationunitid                                             4188 non-null   object 
 4   organisationunitname                                           4188 non-null   object 
 5   organisationunitcode                                           12 non-null     object 
 6   organisationunitdescription                                    0 non-null      float64
 7   105-EP01b. Malaria Total                  